In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import lightgbm as lgb
import joblib

In [2]:
input_image = loadmat('H:\data\hyp_data.mat')['hyp_data']
output_image = loadmat('H:\data\X.mat')['X']

In [3]:
type(input_image)

numpy.ndarray

In [4]:
input_image.shape

(144, 144, 200)

In [5]:
input_image[0].shape

(144, 200)

In [6]:
input_image[0][0,]

array([3172, 4142, 4506, 4279, 4782, 5048, 5213, 5106, 5053, 4750, 4816,
       4769, 4610, 4805, 4828, 4861, 4767, 4624, 4549, 4463, 4462, 4446,
       4445, 4336, 4381, 4319, 4207, 4305, 4311, 3991, 4168, 3942, 4061,
       4365, 4318, 4252, 4869, 5284, 5055, 3591, 5175, 5217, 5058, 4969,
       4721, 4291, 4555, 4886, 4868, 4806, 4783, 4811, 4709, 3903, 3795,
       3715, 3359, 2130, 2269, 2480, 3145, 3626, 4060, 4296, 4211, 4225,
       4157, 4133, 4082, 4048, 3935, 3843, 3784, 3642, 3271, 2707, 1707,
       1564, 1838, 1719, 2229, 2764, 2919, 2873, 2977, 2913, 3034, 3051,
       3124, 3101, 3033, 2713, 2740, 2947, 2706, 2834, 2856, 2683, 2400,
       2229, 1822, 1542, 1097, 1047, 1069, 1100, 1122, 1259, 1365, 1261,
       1374, 1630, 1851, 2028, 2130, 2170, 2205, 2214, 2204, 2100, 2106,
       2146, 2089, 2078, 2134, 2127, 2074, 2057, 2045, 2003, 1999, 1959,
       1924, 1883, 1843, 1781, 1716, 1698, 1645, 1540, 1410, 1294, 1131,
       1044, 1032, 1045, 1100, 1212, 1295, 1244, 11

In [7]:
input_image[0][143,].shape

(200,)

In [8]:
spectral = list()
for i in range(input_image.shape[0]):
    for j in range(input_image.shape[1]):
        spectral.append(input_image[i][j,])
spectral_array = np.array(spectral)

In [9]:
spectral_array.shape

(20736, 200)

In [10]:
pca = PCA(n_components=30)

In [13]:
pca.fit(spectral_array)

PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [14]:
pca_input_iamge = pca.transform(spectral_array)

In [17]:
pca.singular_values_

array([725533.17658989, 422920.09519002, 112020.64143512,  51443.86345089,
        37821.62168121,  33253.2359909 ,  23200.28001581,  16810.66107045,
        16167.06514897,  13138.87550709,  11897.31170378,  10975.55839113,
        10241.67867677,   8096.56704771,   7941.52624596,   7498.19566705,
         7312.96286756,   6856.05559166,   6564.25686992,   6209.17886393,
         6082.79281997,   5993.76680611,   5760.56121782,   5640.72198473,
         5593.57205227,   5517.90156429,   5386.02186576,   5330.09203276,
         5238.25965562,   5167.68884381])

In [19]:
pca.explained_variance_ratio_

array([7.25040461e-01, 2.46356675e-01, 1.72839818e-02, 3.64514530e-03,
       1.97028016e-03, 1.52305476e-03, 7.41368525e-04, 3.89239831e-04,
       3.60006272e-04, 2.37773848e-04, 1.94959935e-04, 1.65920841e-04,
       1.44474095e-04, 9.02920522e-05, 8.68671637e-05, 7.74392662e-05,
       7.36604601e-05, 6.47435265e-05, 5.93497406e-05, 5.31026301e-05,
       5.09628539e-05, 4.94820148e-05, 4.57064281e-05, 4.38245112e-05,
       4.30949285e-05, 4.19368289e-05, 3.99561758e-05, 3.91306540e-05,
       3.77939022e-05, 3.67824288e-05])

In [16]:
pca_input_iamge.shape

(20736, 30)

In [18]:
pca_input_iamge[1]

array([ 5310.14504592, -1874.08190525,   532.9421584 ,  -180.18950009,
         136.78141482,  -168.41546566,    95.28565953,    13.93525059,
           6.87576673,    -7.35003781,    21.69726012,    59.20058991,
          44.01497852,  -116.80147124,   -83.60287417,   -45.0078664 ,
           6.94458093,   -36.40291763,    17.79792474,    41.72823264,
         -39.25229262,    23.7429987 ,    -6.16167569,    21.93371329,
          23.23181569,   -39.26856233,    46.03490913,   -17.6815108 ,
          -7.69952826,   -55.49570591])

## 检验

In [ ]:
# 可以提取出高光谱数据的一个channel的数据 144*144
def split_one_channel(A ,channel):
    d = [0]*144
    for i in range(144):
        d[i] = input_image[i][:,channel-1]
    e = np.vstack((d[i] for i in range(144)))
    return e  
# 参数说明
# channel 通道
# A 高光谱立方块

In [ ]:
# input_image[0][:,0]
input_image[1][:,0]

In [ ]:
d = list()
for i in range(144):
    d.append(input_image[i][:,0])
d_arr = np.array(d)

In [ ]:
d_arr

In [ ]:
s = split_one_channel(input_image,1)
type(d_arr)
type(s)

## 降维后的操作

In [ ]:
pca_input_iamge = pca.transform(spectral_array)

In [ ]:
pca_input_iamge.shape

In [ ]:
pca_input_iamge[20735]

In [ ]:
pca_return_array_one = np.vstack([pca_input_iamge[0:144]])
# pca_return_array = list()
# pca_return_array.ap

In [ ]:
# 注意vstcak分割的用法
pca_return_array = np.vsplit(pca_input_iamge,144)
pca_return_array = np.array(pca_return_array) ########## ￥￥￥￥￥￥￥￥￥￥￥￥￥处理完后的矩阵

In [ ]:
pca_return_array.shape

In [ ]:
pca_return_array[143]

In [ ]:
144*144

# 整体操作

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import joblib

In [2]:
input_image = loadmat('H:\data\hyp_data.mat')['hyp_data']
output_image = loadmat('H:\data\X.mat')['X']

### PCA降维操作

In [8]:
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension)
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,144)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array

### 提取一个channel的空间信息

In [27]:
# 可以提取出高光谱数据的一个channel的数据 144*144
def split_one_channel(A,channel):
    d = [0]*144
    for i in range(144):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(144)))
    return e  
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


# 将镜像处理后的矩阵进行向量化操作
def split_array(A,fil):
    s = []
    length = len(A)-fil+1
    for i in range(length): # row
        for j in range(length): # col
            temp = [0]* fil
            for z in range(fil):
                temp[z] = A[i+z,j:j+fil]
            com = np.concatenate([item for item in temp ])
            s.append(com)
    return s 
# 参数说明
# A 镜像处理后的矩阵
# fil  filter size


# 计算欧几里得距离
def Euclidean(v1,v2):
    return np.linalg.norm(v1-v2)


# 计算单个patch的空间信息
def patch_spitial(v1, K_centroid):
    distance_array = np.array([Euclidean(v1, centroid) for centroid in K_centroid]) #距离matrix
    sums = np.sum(distance_array)
    average = np.average(distance_array)
    spitial_one_patch = np.maximum(0,average - distance_array) #依据公式求空间信息
    return spitial_one_patch

# 对一个channel的求解进行向量化
def channel_spitial(channel_matrix, K_centroid):
    channel_spitial_list = list()
    for i in range(len(channel_matrix)):
        channel_spitial_list.append(patch_spitial(channel_matrix[i], K_centroid))
    return np.array(channel_spitial_list)

### 整体求解

In [5]:
## 构造n个通道的向量
def create_whole_vector(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    whole_split_matrix_array = np.array([split_array(whole_mirror_matrix_array[i],fil) for i in range(dim)])
    return whole_split_matrix_array

## 通过k-means求解 whole_K_centroid
K =5 # K个中心
def create_centroid(K,dim,whole_split_matrix_array):
    whole_kmeans = [KMeans(n_clusters=K, random_state=0).fit(whole_split_matrix_array[i]) for i in range(dim)]
    whole_K_centroid = [whole_kmeans[i].cluster_centers_ for i in range(dim) ]
    return whole_K_centroid

## 求整体空间信息
def create_spe_spi(whole_K_centroid,whole_split_matrix_array):
    whole_temp = list()
    for i in range(len(whole_K_centroid)):
        whole_temp.append(channel_spitial(whole_split_matrix_array[i],whole_K_centroid[i]))
    whole_spitial = np.array(whole_temp)
    return whole_spitial


###### 可选操作，将spectral-spitial信息以numpy形式保存
def save_spe_spi(whole_spitial):
    np.save("whole_spitial.npy",whole_spitial)
###### 可选操作，载入数据
def load_spe_spi(name):
    s = np.load(name)
    return s

## 合成sample_with_lables
def sample_with_lables(s,dim,output_image):
    sample = np.hstack([s[i]for i in range(dim)])
    label = output_image.reshape(len(sample),1)
    sample_lb = np.hstack([sample,label])
    return sample_lb


## 提取出需要分类的数据
def need_claasify(sample_lb):
    sample_lab_fix_list = list()
    for i in range(len(sample_lb)):
        if sample_lb[i,-1] != 0:
            sample_lab_fix_list.append(sample_lb[i,:])
    sample_lab_fix = np.array(sample_lab_fix_list)
    return sample_lab_fix


## 将数据归一化并且存储：
def scaler_save(sample_lab_fix, path = 'H:\data\spec_spi.csv'):
    ## 归一化
    data_D = preprocessing.StandardScaler().fit_transform(sample_lab_fix[:,:-1])
    data_L = sample_lab_fix[:,-1]

    ## 存储
    new = np.column_stack((data_D, data_L))
    new_ = pd.DataFrame(new)
    new_.to_csv(path,header=False,index=False)
    

## 分类数据提取
def classify_pre(path='H:\data\spec_spi.csv'):
    data = pd.read_csv(path,header=None)
    data = data.as_matrix()
    data_D = data[:,:-1]
    data_L = data[:,-1]
    data_train, data_test, label_train, label_test = train_test_split(data_D,data_L,test_size=0.9)
    return data_train, data_test, label_train, label_test

## 分类进行

In [6]:
## 设置初始值
DIM = 30
FIL = 3

In [9]:
pca_return_array = pca_hpi(input_image,DIM) # 降维操作

[7.25040461e-01 2.46356675e-01 1.72839818e-02 3.64514530e-03
 1.97028016e-03 1.52305476e-03 7.41368525e-04 3.89239831e-04
 3.60006272e-04 2.37773848e-04 1.94959934e-04 1.65920845e-04
 1.44474087e-04 9.02926205e-05 8.68673915e-05 7.74392631e-05
 7.36564768e-05 6.47424790e-05 5.93460400e-05 5.31272071e-05
 5.09545156e-05 4.94690855e-05 4.57254339e-05 4.38107329e-05
 4.31095498e-05 4.18094715e-05 4.00067181e-05 3.90565962e-05
 3.84721533e-05 3.71494110e-05]


In [10]:
pca_return_array.shape

(144, 144, 30)

In [11]:
whole_split_matrix_array = create_whole_vector(DIM,FIL,pca_return_array) ## 对30维高光谱数据进行向量化操作处理

In [12]:
whole_split_matrix_array.shape

(30, 20736, 9)

In [13]:
whole_K_centroid = create_centroid(FIL,DIM,whole_split_matrix_array) ## 通过k-means求解空间前置信息

In [14]:
len(whole_K_centroid)

30

In [15]:
whole_spitial = create_spe_spi(whole_K_centroid,whole_split_matrix_array) # 通过欧几里得距离求解空谱信息

In [16]:
whole_spitial.shape  # 验证空谱信息提出成功

(30, 20736, 3)

In [17]:
save_spe_spi(whole_spitial)   ##将空谱信心保存为numpy格式

In [18]:
spec_spi = load_spe_spi('whole_spitial.npy')  # 将空谱信息载入

In [19]:
spec_spi.shape

(30, 20736, 3)

In [20]:
sample_lb = sample_with_lables(spec_spi,DIM,output_image) # 合成带有标签的样本

In [21]:
sample_lb.shape

(20736, 91)

In [22]:
sample_lab_fix = need_claasify(sample_lb) ## 提取需要分类的样本

In [23]:
sample_lab_fix.shape

(10366, 91)

In [24]:
scaler_save(sample_lab_fix, path = 'H:\data\pca_spec_spic_30.csv') # 归一化后保存

In [25]:
data_train, data_test, label_train, label_test = classify_pre(path='H:\data\pca_spec_spic_30.csv') #分类前置操作

#### 分类执行

In [26]:
# clf = lgb.LGBMClassifier()

clf = SVC(kernel='linear')

# # clf = XGBClassifier(max_depth=20,learning_rate=0.3,n_estimators=500,silent=False,
#                        objective='multi:softmax',
#                         min_child_weight=1,
#                         gamma=0.,
#                         scale_pos_weight=1)

# clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.5,
#      max_depth=20, random_state=0)
# clf = AdaBoostClassifier(
#     SVC(kernel='linear'),
#     n_estimators=2,
#     learning_rate=1,
#     algorithm="SAMME")

clf.fit(data_train,label_train)
# pred = clf.predict(data_test)
# accuracy = metrics.accuracy_score(label_test, pred)*100
# print(accuracy)
acc = clf.score(data_test, label_test)
print(acc)
joblib.dump(clf,"pca_specr_spi_30.m")

0.8554126473740622


['pca_specr_spi_30.m']